In [1]:
import pandas as pd
import numpy as np

## Plan
- explore data myself and meet up with jason to construct detailed parsing plna
- current doing EDA

## problem
- how deal with missing value?
- it seems like all_cell_markers includes all the avaliable dat set from the human cell, mouse cell, and single cell, but further validation is needed
- the mapping index
- lab's working schedule for next few weeks

# Exploration on all_cell_markers

insights:
- tissueType: change the undefined into NaN
- UberonOntologyID: two main kinds (UBERON_0001255, CL_0002248)
- CencerType: change "normal" to NaN
- cellType: normal cell or cancer cell
- expected cellMarker == GeneSymbol == geneID == proteinID


In [136]:
all_cell_markers_df = pd.read_csv('../data/all_cell_markers.txt', sep = r'\t')
all_human_markers_df = pd.read_csv('../data/Human_cell_markers.txt', sep = r'\t')
all_mouse_markers_df = pd.read_csv('../data/Mouse_cell_markers.txt', sep = r'\t')
all_singleCell_markers_df = pd.read_csv('../data/Single_cell_markers.txt', sep = r'\t')
# with open('../data/all_cell_markers.txt', 'r') as file:
#     firstline = file.readline()

/opt/anaconda3/envs/scripps_microbiome/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [140]:
all_cell_markers_df.shape

(4123, 15)

In [141]:
all_human_markers_df.shape

(2868, 15)

In [142]:
all_mouse_markers_df.shape

(1255, 15)

In [143]:
all_singleCell_markers_df.shape

(535, 15)

In [144]:
535 + 1255 + 2868 

4658

## Column meaning understand
- speciesType: the species from which the data originates (Human and Mouse)
- tissueType: the type of tissues (181 kinds)
- UberonOntologyID: not sure (ChatGPT: A unique identifier from the Uberon ontology, which is a database that classifies anatomical structures)
- cancerType: association with specific cancer
- cellName: name of the cell type
- CellOntologyID: unique identifier of the cell
- cellMarker: a marker molecule of the cell: in string list, can be convert to list
- geneSymbol: gene associate with the cell marker
- geneID: unique identifier of the gene
- proteinName: name of the protein
- proteinID: unique identifier for the protein
- markerResource: the type of resouce or methologoy used to identify the marker
- PMID: The PudMed ID for the publication or study where the marker data was reported
- Company: the company associated with the resouces


In [34]:
all_cell_markers_df.columns

Index(['speciesType', 'tissueType', 'UberonOntologyID', 'cancerType',
       'cellType', 'cellName', 'CellOntologyID', 'cellMarker', 'geneSymbol',
       'geneID', 'proteinName', 'proteinID', 'markerResource', 'PMID',
       'Company'],
      dtype='object')

## Data distribition

In [36]:
all_cell_markers_df.speciesType.value_counts()

Human    2868
Mouse    1255
Name: speciesType, dtype: int64

In [ ]:
all_cell_markers_df.tissueType.value_counts()

Undefined           695
Brain               254
Bone marrow         254
Lung                249
Liver               240
                   ... 
Fetal brain           1
Vocal fold            1
Nerve                 1
Primitive streak      1
Premolar              1
Name: tissueType, Length: 181, dtype: int64

In [46]:
all_cell_markers_df.UberonOntologyID.value_counts()

UBERON_0000955    254
UBERON_0002371    254
UBERON_0002048    249
UBERON_0002107    240
UBERON_0005408    215
                 ... 
UBERON_0003215      1
UBERON_0006530      1
UBERON_0001052      1
UBERON_0003922      1
UBERON_0005384      1
Name: UberonOntologyID, Length: 172, dtype: int64

In [47]:
all_cell_markers_df.cancerType.value_counts()

Normal                                      3315
Breast Cancer                                100
Colorectum Cancer                             36
Prostate Cancer                               35
Colon Cancer                                  34
                                            ... 
Non-Hodgkin's Lymphoma                         1
Non-small Cell Lung Cancer (circulating)       1
Gonadoblastoma                                 1
Vascular Tumour                                1
oligodendroglioma                              1
Name: cancerType, Length: 133, dtype: int64

In [48]:
all_cell_markers_df.cellType.value_counts()

Normal cell    3315
Cancer cell     808
Name: cellType, dtype: int64

In [49]:
all_cell_markers_df.cellName.value_counts()

Cancer stem cell                                             593
Stem cell                                                    186
Endothelial cell                                             157
Macrophage                                                   139
Mesenchymal stem cell                                        137
                                                            ... 
Neuron-restricted precursor                                    1
Limbal mesenchymal cell                                        1
Quiescent small intestinal stem cell                           1
Adipose multilineage-differentiating stress-enduring cell      1
CD4+ T follicular helper cell                                  1
Name: cellName, Length: 681, dtype: int64

In [50]:
all_cell_markers_df.CellOntologyID.value_counts()

CL_0000034    308
CL_0000134    181
CL_0000115    168
CL_0000235    141
CL_0000084    132
             ... 
CL_0000166      1
CL_0008020      1
CL_0000058      1
CL_0000210      1
CL_0000850      1
Name: CellOntologyID, Length: 281, dtype: int64

In [51]:
all_cell_markers_df.cellMarker.value_counts()

CD133                                                                 109
CD44                                                                   58
CD31                                                                   50
CD68                                                                   47
CD3                                                                    39
                                                                     ... 
BDCA1, CD11c                                                            1
BDCA2, CD123                                                            1
CLEC9A                                                                  1
TAGLN2                                                                  1
ASCL1, BOC, CCND2, CD24, CHD7, EGFR, NFIB, SOX11, SOX2, SOX4, TCF4      1
Name: cellMarker, Length: 2517, dtype: int64

In [52]:
all_cell_markers_df.geneSymbol.value_counts()

PROM1                                                                 109
CD44                                                                   54
PECAM1                                                                 45
CD68                                                                   39
CD34                                                                   30
                                                                     ... 
KRT18, KRT19                                                            1
LIN28A                                                                  1
PROM1, NCAM1                                                            1
CLEC4F, VSIG4                                                           1
ASCL1, BOC, CCND2, CD24, CHD7, EGFR, NFIB, SOX11, SOX2, SOX4, TCF4      1
Name: geneSymbol, Length: 2485, dtype: int64

In [53]:
all_cell_markers_df.geneID.value_counts()

8842                                                                     109
960                                                                       54
5175                                                                      45
968                                                                       39
947                                                                       30
                                                                        ... 
8842, 4684                                                                 1
165530, 11326                                                              1
59, 2670                                                                   1
Albumin family, 4311, 3875, 3856                                           1
429, 91653, 894, 100133941, 55636, 1956, 4781, 6664, 6657, 6659, 6925      1
Name: geneID, Length: 2487, dtype: int64

In [54]:
all_cell_markers_df.proteinName.value_counts()

PROM1                                                                 115
CD44                                                                   61
PECA1                                                                  57
CD68                                                                   47
[CD3D, CD3E, CD3G]                                                     39
                                                                     ... 
Albumin family, NEP, K1C18, K2C8                                        1
Albumin family, K2C8                                                    1
Albumin family, K1C18                                                   1
FETA, FBF1, HNF4A                                                       1
ASCL1, BOC, CCND2, CD24, CHD7, EGFR, NFIB, SOX11, SOX2, SOX4, ITF2      1
Name: proteinName, Length: 2350, dtype: int64

In [55]:
all_cell_markers_df.proteinID.value_counts()

O43490                                                                                    109
P16070                                                                                     54
P16284                                                                                     45
P34810                                                                                     39
P28906                                                                                     30
                                                                                         ... 
Q8N1N0, Q9Y279                                                                              1
P62736, P14136                                                                              1
Albumin family, P08473, P05783, P05787                                                      1
Albumin family, P05787                                                                      1
P50553, Q9BWV1, P30279, P25063, Q9P2D1, P00533, O00712, P357

In [56]:
all_cell_markers_df.markerResource.value_counts()

Experiment                3064
Single-cell sequencing     535
Company                    355
Review                     169
Name: markerResource, dtype: int64

In [57]:
all_cell_markers_df.PMID.value_counts()

Company     355
30018341     41
30093597     37
29802404     27
29545511     21
           ... 
28869524      1
17714779      1
18701045      1
20060164      1
16849681      1
Name: PMID, Length: 2379, dtype: int64

In [58]:
all_cell_markers_df.Company.value_counts()

ebioscience           77
miltenyibiotec        60
abcam                 53
biolegend             52
bio-rad-antibodies    47
rndsystems            35
labome                16
bdbiosciences         15
Name: Company, dtype: int64

## Missing value

In [61]:
all_cell_markers_df.isna().mean().sort_values(ascending=False)

Company             0.913898
CellOntologyID      0.210526
UberonOntologyID    0.180936
proteinName         0.013582
proteinID           0.013582
geneSymbol          0.012127
geneID              0.012127
speciesType         0.000000
tissueType          0.000000
cancerType          0.000000
cellType            0.000000
cellName            0.000000
cellMarker          0.000000
markerResource      0.000000
PMID                0.000000
dtype: float64

In [132]:
test = all_cell_markers_df.copy().dropna(subset=['geneSymbol', 'proteinID'])    #[['cellMarker', 'geneSymbol', 'geneID']]
test['cellMarker'] = test['cellMarker'].apply(lambda x: x.split(', '))
test['geneSymbol'] = test['geneSymbol'].apply(lambda x: x.split(', '))
test['geneID'] = test['geneID'].apply(lambda x: x.split(', '))
test['proteinID'] = test['proteinID'].apply(lambda x: x.split(', '))

test['len_cellMarker'] = test['cellMarker'].apply(len)
test['len_geneSymbol'] = test['geneSymbol'].apply(len)
test['len_geneID'] = test['geneID'].apply(len)
test['len_proteinID'] = test['proteinID'].apply(len)

In [115]:
x = test[~((test['len_cellMarker'] == test['len_geneSymbol']) & (test['len_geneSymbol'] == test['len_geneID']))]

In [122]:
x.iloc[0]['geneSymbol']

['ITGAM', 'CD14', 'ITGB2', '[FCGR2A', 'FCGR2B', 'FCGR2C]', 'CD68']

In [123]:
x.iloc[0]['cellMarker']

['CD11b', 'CD14', 'CD18', 'CD32', 'CD68']

In [124]:
x.iloc[0]['geneID']

['3684', '929', '3689', '[2212', '2213', '9103]', '968']

In [129]:
# the problem of spliting for the cell maker

In [130]:
test

,speciesType,tissueType,UberonOntologyID,cancerType,cellType,cellName,CellOntologyID,cellMarker,geneSymbol,geneID,proteinName,proteinID,markerResource,PMID,Company,len_cellMarker,len_geneSymbol,len_geneID
0,Human,Kidney,UBERON_0002113,Normal,Normal cell,Proximal tubular cell,NaN,[Intestinal Alkaline Phosphatase],[ALPI],[248],PPBI,P09923,Experiment,9263997,NaN,1,1,1
1,Human,Liver,UBERON_0002107,Normal,Normal cell,Ito cell (hepatic stellate cell),CL_0000632,[Synaptophysin],[SYP],[6855],SYPH,P08247,Experiment,10595912,NaN,1,1,1
2,Human,Endometrium,UBERON_0001295,Normal,Normal cell,Trophoblast cell,CL_0000351,[CEACAM1],[CEACAM1],[634],CEAM1,P13688,Experiment,10751340,NaN,1,1,1
3,Human,Germ,UBERON_0000923,Normal,Normal cell,Primordial germ cell,CL_0000670,[VASA],[DDX4],[54514],DDX4,Q9NQI0,Experiment,10920202,NaN,1,1,1
4,Human,Corneal epithelium,UBERON_0001772,Normal,Normal cell,Epithelial cell,CL_0000066,[KLF6],[KLF6],[1316],KLF6,Q99612,Experiment,12407152,NaN,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4118,Human,Embryo,UBERON_0000922,Normal,Normal cell,8-cell stage cell (Blastomere),CL_0000353,"[C11orf48, C19orf53, DHX9, DIABLO, EIF1AD, EIF...","[LBHD1, C19orf53, DHX9, DIABLO, EIF1AD, EIF4G1...","[79081, 28974, 1660, 56616, 84285, 1981, 26017...","LBHD1, L10K, DHX9, DBLOH, EIF1A, IF4G1, FA32A,...","Q9BQE6, Q9UNZ5, Q08211, Q9NR28, Q8N9N8, Q04637...",Single-cell sequencing,23892778,NaN,19,19,19
4119,Mouse,Embryo,UBERON_0000922,Normal,Normal cell,8-cell stage cell (Blastomere),CL_0000353,"[Abcf1, Cdc37, Diablo, Eif1ad, Eif4g1, Fam32a,...","[Abcf1, Cdc37, Diablo, Eif1ad, Eif4g1, Fam32a,...","[224742, 12539, 66593, 69860, 208643, 67922, 7...","ABCF1, CDC37, DBLOH, EIF1A, IF4G1, FA32A, KDM5...","Q6P542, Q61081, Q9JIQ3, Q3THJ3, Q6NZJ6, Q9CR80...",Single-cell sequencing,23892778,NaN,12,12,12
4120,Human,Embryo,UBERON_0000922,Normal,Normal cell,Morula cell (Blastomere),CL_0000360,"[ADCK1, AGL, AIMP1, AKAP12, ARPC3, ATP1B3, ATP...","[ADCK1, AGL, AIMP1, AKAP12, ARPC3, ATP1B3, NA,...","[57143, 178, 9255, 9590, 10094, 483, NA, 586, ...","ADCK1, GDE, AIMP1, AKA12, ARPC3, AT1B3, AT5F1,...","Q86TW2, P35573, Q12904, Q02952, O15145, P54709...",Single-cell sequencing,23892778,NaN,112,112,112
4121,Mouse,Embryo,UBERON_0000922,Normal,Normal cell,Morula cell (Blastomere),CL_0000360,"[Aimp1, Atp5f1, Atp5h, Bcat1, Bin1, Ccbl2, Cct...","[Aimp1, Atp5f1, Atp5h, Bcat1, Bin1, Kyat3, Cct...","[13722, 11950, 71679, 12035, 30948, 229905, 12...","AIMP1, AT5F1, ATP5H, BCAT1, BIN1, KAT3, TCPE, ...","P31230, Q9CQQ7, Q9DCX2, P24288, O08539, Q71RI9...",Single-cell sequencing,23892778,NaN,37,37,37
